In [ ]:
import networkx as nx
import random
import json
import plotly.graph_objects as go
import numpy as np
#from google.colab import drive
import os

# Function to generate graphs with vertices and edges based on the provided packages
def generate_graph(node, range_R, range_B, range_L):
    G = nx.Graph()

    # Generate vertices with R and B values
    for i in range(node):
        R = random.randint(*range_R)
        B = random.randint(*range_B)
        G.add_node(i, R=R, B=B)

    # Generate edges with L values ​​(randomly generated)
    for i in range(node):
        for j in range(i+1, node):
            L = random.randint(*range_L)
            G.add_edge(i, j, L=L)

    return G

# Function to find the shortest path (minimum spanning tree) using Kruskal's algorithm
def kruskal_minimum_path(G):
    mst = nx.minimum_spanning_tree(G, weight='L', algorithm='kruskal')
    return mst

# Function that generates the intermediate graph including the Kruskal path
def generate_intermediate_graph(G, mst):
    # Copy the complete graph G
    G_intermediate = G.copy()

    # Keeps the edges of the graph minimal (Kruskal)
    mst_edges = list(mst.edges(data=True))

    # Removes random edges from the original graph, ensuring that MST edges remain
    all_edges = list(G_intermediate.edges(data=True))

    # Sets the percentage of edges to keep beyond MST (Kruskal)
    p_edges = 0.15 # Keeps 15% of the edges random, from the complete graph G (adjust as you prefer)
    num_edges_to_keep = int(p_edges * len(all_edges))

    # Selects random edges to keep (in addition to the MST ones)
    edges_to_keep = random.sample(all_edges, num_edges_to_keep)

    # Ensures that the MST edges are in the intermediate graph
    edges_to_keep.extend(mst_edges)

    # Create a new intermediate graph with these edges
    G_intermediate.clear_edges()
    G_intermediate.add_edges_from(edges_to_keep)

    return G_intermediate

# Function to generate random 3D coordinates for graph nodes
def generate_3D_coordinates(G):
    # Número de nós no grafo
    num_nodes = len(G.nodes)

    # Generates random coordinates for each node, within a 100x100x100 cube
    pos = {}
    for node in G.nodes:
        x = np.random.uniform(0, 100)
        y = np.random.uniform(0, 100)
        z = np.random.uniform(0, 100)
        pos[node] = (x, y, z)  # Armazena as coordenadas 3D para o nó

    return pos

# Function to convert the graph to random 3D coordinates
# These values ​​are random coordinates generated for each node in 3D space.
# The function generate_3D_coordinates calculates the coordinates and stores them in the
# dictionary pos, where each node receives a tuple containing the coordinates; x, y and z.
def display_3D_graph(G, pos, title="3D Graph"):
    edge_x = []
    edge_y = []
    edge_z = []
    edge_text = []  # List to store latency texts

    # Processing the edges
    for edge in G.edges(data=True):
        x0, y0, z0 = pos[edge[0]]
        x1, y1, z1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

        # Calculating the midpoint of an edge
        mx, my, mz = (x0 + x1) / 2, (y0 + y1) / 2, (z0 + z1) / 2
        edge_text.append(f"L: {edge[2]['L']:.2f}")  # Adicionando a latência

    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        line=dict(width=2, color='gray'),
        hoverinfo='none',
        mode='lines',
        name='Edges')

    # Processing the vertices
    node_x = []
    node_y = []
    node_z = []
    node_text = []

    for node in G.nodes(data=True):
        x, y, z = pos[node[0]]
        node_x.append(x)
        node_y.append(y)
        node_z.append(z)
        node_text.append(f"<b>N</b>: {node[0]}<br><b>R</b>: {node[1]['R']}<br><b>B</b>: {node[1]['B']}")

    node_trace = go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode='markers+text',
        textposition="middle center",  # To display attributes in the middle of nodes (vertices)
        hoverinfo='text',
        marker=dict(
            showscale=False,
            color='lightgreen',
            size=45,   # Size of the circle representing the nodes (vertices)
            line_width=1.0),
        text=node_text,
        name='Active Nodes')

    # Display latencies at the midpoint of edges
    edge_text_trace = go.Scatter3d(
        x=[(pos[edge[0]][0] + pos[edge[1]][0]) / 2 for edge in G.edges()],
        y=[(pos[edge[0]][1] + pos[edge[1]][1]) / 2 for edge in G.edges()],
        z=[(pos[edge[0]][2] + pos[edge[1]][2]) / 2 for edge in G.edges()],
        mode='text',
        text=[f"<b>L</b>: {G.edges[edge]['L']}" for edge in G.edges()],
        hoverinfo='text',
        textposition="middle center",
        name='Latencies'
    )

    # Set the chart title
    title = title

    # Create the layout with the title centered and include the captions in a list of notes
    annotations = [
        dict(
            text="<b>N</b> = Node<br><b>R</b> = Resource<br><b>B</b> = Bandwidth<br><b>L</b> = Latency",  # Create the layout with the title centered and include the captions in a list of notes
            showarrow=False,  # No arrow
            xref="paper", yref="paper",  # Reference to the total area of ​​the graph (not the 3D scene)
            x=0.9, y=1.0,  # Adjust as needed
            xanchor='left',  # Align left
            yanchor='top',  # Positions the caption at the top of the area
            align='left',  # Ensures left alignment
            font=dict(size=14, color="black")
        )
    ]

    layout = go.Layout(
        title=dict(
            text=title,
            x=0.5,
            xanchor='center',
            font=dict(size=20)
        ),
        showlegend=True,  # Enable subtitles
        scene=dict(
            xaxis=dict(showbackground=False, visible=False),
            yaxis=dict(showbackground=False, visible=False),
            zaxis=dict(showbackground=False, visible=False)
        ),
        margin=dict(l=0, r=0, b=100, t=100),  # Adjusting the bottom and top margins
        width=1600,
        height=1200,
        annotations=annotations  # Assign the entire notes list to the layout
    )

    # Create the 3D graph figure
    fig = go.Figure(data=[edge_trace, node_trace, edge_text_trace], layout=layout)

    # Display the chart
    fig.show()

def get_positive_integer(prompt, default):
    while True:
        try:
            user_input = input(f"{prompt} ({default}): ")
            # If the user does not enter anything, it uses the default value.
            value = int(user_input) if user_input else default

            if value > 0:
                return value  # Returns the value if it is positive
            else:
                print("Erro: The number must be a positive value. Please try again.")
        except ValueError:  # Checks if value is an integer
            print("Erro: Invalid input. Please enter an integer.")

import json

def gerar_lista_adjacencias(G_intermediate, output_path):
    adjList = []
    num_nos = max(max(u, v) for u, v in G_intermediate.edges()) + 1  # Find the largest node index
    for i in range(num_nos):
        adjList.append([])

    for u, v, data in G_intermediate.edges(data=True):
        adjList[u].append([v, data['L']])
        adjList[v].append([u, data['L']])  # Undirected graph, add to both nodes

    # Saving the adjacency list in the desired format to the output.json file
    # 'a': Opens the file for writing, starting at the end of the file. Does not erase existing content.
    with open(output_path, 'a') as outfile:
        outfile.write('  "adjList": [\n')

        # Iterate over neighbors and add comma only where needed
        for i, neighbors in enumerate(adjList):
            if i < len(adjList) - 1:
                outfile.write(f"    {json.dumps(neighbors)},\n")  # Adds comma after each element except the last one
            else:
                outfile.write(f"    {json.dumps(neighbors)}\n")   # The last element does not receive a comma

        outfile.write("  ]\n")
        outfile.write("}")

# Main Function
def main():
    #-------------- Values ​​assigned to the IoT network -------------------------
    # EN =  Network Edge, RN = Resource Network
    # Stores the number of edge layer nodes
    edges_nodes = []
    # Setting default values
    default_node = 15
    default_EN = (7, 14)
    default_RN = (20, 300)
    default_BN = (20, 3000)
    default_LN = (2, 100)

    print("---------------- IoT Network Parameters ---------------------------\n")
    # Collecting the number of IoT network nodes
    numNodes = get_positive_integer("Number of nodes: ", default_node)

    # Collecting the number of nodes that belong to the edge layer
    input_EN = input(f"Range for edge nodes (ex: {default_EN[0]} {default_EN[1]}): ")
    range_EN = tuple(map(int, input_EN.split())) if input_EN else default_EN
    # Generating a list for edge_nodes with numbers in the (default) range of 7 to 14, inclusive
    edge_nodes = list(range(range_EN[0], range_EN[1] + 1))

    input_RN = input(f"Range for Resource (R) (ex: {default_RN[0]} {default_RN[1]}): ")
    range_RN = tuple(map(int, input_RN.split())) if input_RN else default_RN

    input_BN = input(f"Range for Bandwidth (B) (ex: {default_BN[0]} {default_BN[1]}): ")
    range_BN = tuple(map(int, input_BN.split())) if input_BN else default_BN

    input_LN = input(f"Range for Latency (L) (ex: {default_LN[0]} {default_LN[1]}): ")
    range_LN = tuple(map(int, input_LN.split())) if input_LN else default_LN

    print("-------------------------------------------------------------------")

    # Generate the graph based on the given parameters
    G = generate_graph(numNodes, range_RN, range_BN, range_LN)

    # Calculate the minimum spanning tree (Kruskal)
    mst = kruskal_minimum_path(G)

    # Generate the random intermediate graph + Kruskal
    G_intermediate = generate_intermediate_graph(G, mst)

    # Generate 3D coordinates for graphs
    pos = generate_3D_coordinates(G)

    # Display the complete graph
    #display_3D_graph(G, pos, title="Complete 3D Graph")
    #print(f"G_complete: {G}")

    # Display the intermediate graph
    display_3D_graph(G_intermediate, pos, title="IoT Network")
    print(f"G_intermediate: {G_intermediate}")

    # Display the minimum spanning tree (Kruskal)
    #display_3D_graph(mst, pos, title="Minimum Spanning Tree (Kruskal) 3D")
    #print(f"G_mst: {mst}")

    # Generate 15 random values ​​for R and B (from the network) within the specified ranges
    RN_values = [G_intermediate.nodes[_]['R'] for _ in range(numNodes)]
    BN_values = [G_intermediate.nodes[_]['B'] for _ in range(numNodes)]

    # Initial values ​​of vectors: V_Busy and V_Inactive
    V_Busy = np.zeros(numNodes).astype(int)  # Converting values ​​to integers
    V_Inactive = np.zeros(numNodes).astype(int)  # Converting values ​​to integers
    #---------------------------------------------------------------------------
    #----------------------- Values ​​assigned to Jobs ---------------------------
    # EJ = Edge Job, RJ = Resource Job
    # Setting default values
    default_job = 1
    default_RJ = (10, 350)
    default_BJ = (10, 2500)
    default_LJ = (1, 250)
    print("---------------------- Job Parameters -----------------------------\n")
    # Collecting the number of jobs to be executed on the IoT network
    numJob = get_positive_integer("Number of jobs: ", default_job)

    # Generate job values ​​randomly
    input_RJ = input(f"Range for Resource (R) (ex: {default_RJ[0]} {default_RJ[1]}): ")
    range_RJ = tuple(map(int, input_RJ.split())) if input_RJ else default_RJ

    input_BJ = input(f"Range for Bandwidth (B) (ex: {default_BJ[0]} {default_BJ[1]}): ")
    range_BJ = tuple(map(int, input_BJ.split())) if input_BJ else default_BJ

    input_LJ = input(f"Range for Latency (L) (ex: {default_LJ[0]} {default_LJ[1]}): ")
    range_LJ = tuple(map(int, input_LJ.split())) if input_LJ else default_LJ


    # Running random.randint numJob times and storing the values ​​in a list
    jr = [random.randint(range_RJ[0], range_RJ[1]) for _ in range(numJob)]

    # Displaying the results for jr
    print(f"Generated jr values: {jr}")
    # Running random.randint numJob times and storing the values ​​in a list
    jb = [random.randint(range_BJ[0], range_BJ[1]) for _ in range(numJob)]

    # Displaying the results for jb
    print(f"Generated jb values: {jb}")

    # Running random.randint numJob times and storing the values ​​in a list
    jl = [random.randint(range_LJ[0], range_LJ[1]) for _ in range(numJob)]

    # Displaying the results for jl
    print(f"Generated jl values: {jl}")

    # Running random.randint numJob times and storing the values ​​in a list
    jo = [random.choice(edge_nodes) for _ in range(numJob)]

    # Displaying the results for jo
    print(f"Generated jo values: {jo}")
    print("-------------------------------------------------------------------")

    #---------------------------------------------------------------------------
    # Mount Google Drive
    #drive.mount('/content/drive')

    # Set the directory path where the .json files are located

    # Setting default values
    default_out_f = r'C:\Users\jonat\OneDrive\Área de Trabalho\teste\input'

    # Prompts user for directory path or uses default value if ENTER is pressed
    output_folder = input(f"Enter a directory path or press ENTER for default (ex: {default_out_f}): ") or default_out_f

    # Check if the output folder exists, if not, create it
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Saving data to a JSON file at the specified path
    output_path = f'C:\\Users\\jonat\\OneDrive\\Área de Trabalho\\Jonatas\\input\\input_{numNodes}_nodes_{numJob}_jobs.json'

    # Initial node values ​​regarding the status: free/busy ------------------
    # The values ​​represent: 0 (free node) and 1 (busy node) ------------------------
    #V_Busy = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    #-------------------------------------------------------------------------------
    # Initial values ​​of the nodes regarding the state: Active/Inactive -------------
    # The values ​​represent: 0 (Active node) and 1 (Inactive node) ------------------
    #V_Inactive = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    #-------------------------------------------------------------------------------

    # Formatting data for printing in the requested format
    formatted_output = (
        '{\n'  # Adds the opening key of the JSON object
        f'  "edge_nodes": {edge_nodes},\n'
        f'  "jr": [{", ".join(map(str, jr))}],\n'
        f'  "jb": [{", ".join(map(str, jb))}],\n'
        f'  "jl": [{", ".join(map(str, jl))}],\n'
        f'  "jo": [{", ".join(map(str, jo))}],\n'
        f'  "V_R": [{", ".join(map(str, RN_values))}],\n'
        f'  "V_B": [{", ".join(map(str, BN_values))}],\n'
        f'  "V_Busy": [{", ".join(map(str, V_Busy))}],\n'
        f'  "V_Inactive": [{", ".join(map(str, V_Inactive))}],\n'
    )

    # Saving formatted output to JSON file
    with open(output_path, "w") as json_file:
        json_file.write(formatted_output)

    # Running the function to generate the adjacency list
    gerar_lista_adjacencias(G_intermediate, output_path)


if __name__ == "__main__":
    main()
